In [7]:
from socket import *
from io import BytesIO

if True:
    s =socket()
    s.connect(("tc-projecti-zdir.zulong.com", 52002)) # zh_ios_formal
    stream = s.makefile('rb')


In [24]:
if True:
    s =socket()
    s.connect(("tc-projecti-zdir.zulong.com", 52002)) # zh_ios_formal
    stream = s.makefile('rb')

def read_uint(read):
    b0 = read(1)[0]
    if b0 & 0x80 == 0: # 0XXXXXXX
        return b0
    elif b0 & 0x40 == 0: # 10XXXXXX XXXXXXXX
        return int.from_bytes(bytes([b0 & 0x7f]) + read(1), 'big')
    elif b0 & 0x20 == 0: # 110XXXXX XXXXXXXX XXXXXXXX XXXXXXXX
        return int.from_bytes(bytes([b0 & 0x3f]) + read(3), 'big')
    else: # 111XXXXX XXXXXXXX XXXXXXXX XXXXXXXX
        return int.from_bytes(read(4), 'big')

def read_packet(stream):
    pktID = read_uint(lambda n: stream.read(n))
    pktLen = read_uint(lambda n: stream.read(n))
    pktContent = stream.read(pktLen)
    return pktID, pktContent

def read_octet(pktstream):
    bufLen = read_uint(lambda n: pktstream.read(n))
    buf = pktstream.read(bufLen)
    return buf

pktID, pktContent = read_packet(stream)
assert pktID == 7101

import zlib
def parseDirInfo(pktstream):
    serverList = read_octet(pktstream)
    version = read_octet(pktstream)
    versionList = read_octet(pktstream)
    
    serverListLength = int.from_bytes(pktstream.read(4), 'big')
    versionLength = int.from_bytes(pktstream.read(4), 'big')
    versionListLength = int.from_bytes(pktstream.read(4), 'big')
    
    serverList = zlib.decompress(serverList)[:serverListLength]
    version = zlib.decompress(version)[:versionLength]
    versionList = zlib.decompress(versionList)[:versionListLength]
    return {
        'serverList': serverList.decode(), 
        'version': version.decode(), 
        'versionList': versionList.decode(),
    }

dirInfo = parseDirInfo(BytesIO(pktContent))

{'serverList': '<serverlist version="1" name="以闪亮之名">\n'
               '  <servergroup name="内部测试" os="pc,iOS,Android" auth="auany" '
               'evaluation="false" accountwithzoneid="true" '
               'channel="zulong,qq,wechat" grc_url="127.0.0.1:6060" '
               'gda_url="gda-test.zulong.com:308000" '
               'ssp_url="ssp-test-longzu.zulong.com:3080" '
               'first_opentime="1617873855">\n'
               '    <server zoneid="9001" name="预发布" address="42.192.67.15" '
               'port="9227" weight="0" load="7" closed="true" hidden="true" '
               'new="false" evaluation="false" '
               'notice="《以闪亮之名》将于3月24日早上6点正式开服，期待与您在闪亮世界中相会！" is_center="0" '
               'nochannel="" support_multi_system_plats="false" area="0" '
               'crashed="false"/>\n'
               '    <server zoneid="9002" name="obTest" address="121.5.229.72" '
               'port="9227" weight="0" load="4" closed="true" hidden="true" '
               '

In [25]:
for c in dirInfo:
    print(c, dirInfo[c])

serverList <serverlist version="1" name="以闪亮之名">
  <servergroup name="内部测试" os="pc,iOS,Android" auth="auany" evaluation="false" accountwithzoneid="true" channel="zulong,qq,wechat" grc_url="127.0.0.1:6060" gda_url="gda-test.zulong.com:308000" ssp_url="ssp-test-longzu.zulong.com:3080" first_opentime="1617873855">
    <server zoneid="9001" name="预发布" address="42.192.67.15" port="9227" weight="0" load="7" closed="true" hidden="true" new="false" evaluation="false" notice="《以闪亮之名》将于3月24日早上6点正式开服，期待与您在闪亮世界中相会！" is_center="0" nochannel="" support_multi_system_plats="false" area="0" crashed="false"/>
    <server zoneid="9002" name="obTest" address="121.5.229.72" port="9227" weight="0" load="4" closed="true" hidden="true" new="true" evaluation="false" notice="《以闪亮之名》将于3月24日早上6点正式开服，期待与您在闪亮世界中相会！" is_center="0" nochannel="" support_multi_system_plats="false" area="0" crashed="false"/>
  </servergroup>
  <servergroup name="闪亮1区" os="iOS,Android,pc" auth="auany" evaluation="false" accountwithzoneid